In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from control import matlab
from scipy import linalg

### パラメータ設定 ###
Rw = 0.021 # 車輪半径
l = 0.135 # 重心長さ
M = 0.21 # 本体重量
m = 0.0053 # 車輪重量
Jb = M*l**2 # 本体慣性モーメント
Jw = m*Rw**2/2.0 # 車輪慣性モーメント
fm = 4.91e-7 # モータ軸粘性摩擦係数
g = 9.81 # 重力加速度
n = 21 # ギア比
Kt = 1.15e-3 # モータトルク係数

### 状態方程式の記述 ###
E = np.array([[(m + M)*Rw**2 + Jw, M*Rw*l],
            [M*Rw*l, M*l*l + Jb]])
F = np.array([[2*fm, -2*fm],
    [-2*fm, 2*fm]])
G = np.array([[0, 0],
    [0, -M*g*l]])
H = np.array([[-2*n*Kt], [2*Kt]])

A1 = np.hstack((np.zeros((2, 2)), np.eye(2)))
A2 = np.hstack((-np.dot(linalg.inv(E), G), -np.dot(linalg.inv(E), F)))
A = np.vstack((A1, A2))
B = np.vstack((np.zeros((2, 1)), np.dot(linalg.inv(E), H)))
C = np.eye(4)

# A行列の固有値を確認
EigVaA, EigVeA = linalg.eig(A)
print("EigenValue (A): ", EigVaA)

### ゲイン設定，どちらかを用いる ###
# ----------------------------------
# 極配置
p = [-...+...j, -...-...j, -...+...j, -...-...j]
# 左2つ・右2つがそれぞれペアの共役複素数になるように設定（例：[-1+1j, -1-1j, -2+2j, -2-2j]）
K = matlab.place(A, B, p) # ゲイン

# 最適制御
Q = np.diag([..., ..., ..., ...])
R = np.array([...])
P = linalg.solve_continuous_are(A, B, Q, R)
K = np.dot(B.T, P)/R # ゲイン

print("Gain: ", K)
# ----------------------------------

# A行列の固有値を確認
EigVaABK, EigVeABK = linalg.eig(A - np.dot(B, K))
print("EigenValue (A-BK): ", EigVaABK)

In [ ]:
### 時系列にそって状態を計算 ###
x = np.array([[0.0], [0.1], [0.0], [0.0]]) # 現在の状態
start = 0.0
dt = 1e-4
tf = 5.0

X = np.zeros((int(tf/dt)+1, 6)) # 保存用変数
X[0, :] = np.hstack((np.array([0.0]), x.T[0], np.array([0.0])))
count = 1

for t in [ x*dt for x in range(int(tf/dt))] :
    u = -np.dot(K, x)
    xdot = np.dot(A, x) + np.dot(B, u)
    x = x + np.dot(xdot, dt)
    X[count, :] = np.hstack((np.array([t]), x.T[0], u[0]))
    count += 1

# データの最大値の表示
MaxTheta = np.max(np.abs(X[:, 1]))
MaxPsi = np.max(np.abs(X[:, 2]))
MaxU = np.max(np.abs(X[:, 5]))
print("Maximum Wheel Angle: ", MaxTheta, "[rad]")
print("Maximum Body Angle: ", MaxPsi, "[rad]")
print("Maximum Input: ", MaxU, "[A]")

In [ ]:
### プロット ###
# Wheel Angle
plt.plot(X[:, 0], X[:, 1])
plt.xlabel("Time [s]")
plt.ylabel(r"Wheel Angle $\theta$ [rad]")
plt.grid(which="major", color="grey", linestyle="--")
plt.savefig("WheelPlot.png")
plt.show()

In [ ]:
# Body Angle
plt.plot(X[:, 0], X[:, 2])
plt.xlabel("Time [s]")
plt.ylabel(r"Body Angle $\psi$ [rad]")
plt.grid(which="major", color="grey", linestyle="--")
plt.savefig("BodyPlot.png")
plt.show()